In [ ]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

# Function to set environment variables
def set_env_variables():
    os.environ['LANGCHAIN_TRACING_V2'] = "true"
    os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
    os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
    os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY', '')

    os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY', '')
    os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACEHUB_API_TOKEN', '')

    print("✅ Environment variables set successfully!")

# Call the function
set_env_variables()


✅ Environment variables set successfully!


In [20]:
## import librairies
from langchain.document_loaders import PyPDFLoader, CSVLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate 
from langchain_groq import ChatGroq
from langchain_core.pydantic_v1 import BaseModel, Field , ConfigDict
from langchain_core.output_parsers import StrOutputParser

# from openai import RateLimitError
#from langchain_core.exceptions import LLMError
from typing import List
from rank_bm25 import BM25Okapi
import fitz
import asyncio
import random
import textwrap
import numpy as np
from enum import Enum

In [2]:
llm = ChatGroq(model_name='Gemma2-9b-it')
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en')

/tmp/ipykernel_2089/2254272975.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en')


In [3]:
# Urls
urls = [
    "https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-2-reflection/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-4-planning/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io"
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splitted_docs = text_splitter.split_documents(docs_list)

# Vector Store
v_store = FAISS.from_documents(splitted_docs, embeddings)
retriever = v_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":4}
)

In [4]:
question = "What are the different kind of agentic design patterns?"
answers = retriever.invoke(question)
print(answers)

[Document(id='689584f8-2fd2-483a-92c5-22fb958f0bac', metadata={'source': 'https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io', 'title': 'Four AI Agent Strategies That Improve GPT-4 and GPT-3.5 Performance', 'description': 'I think AI agent workflows will drive massive AI progress this year — perhaps even more than the next generation of foundation models. This is an important...', 'language': 'en'}, page_content='up tasks and discussing and debating ideas, to come up with better solutions than a single agent would.Next week, I’ll elaborate on these design patterns and offer suggested readings for each.Keep learning!AndrewRead "Agentic Design Patterns Part 2: Reflection"Read "Agentic Design Patterns Part 3, Tool Use"Read "Agentic Design Patterns Part 4: Planning"Read "Agentic Design Patterns Part 5: Multi-Agent Collaboration"ShareSubscribe to The BatchStay updated with weekly AI News and Insights'), Document(id='5259b3a0-4ff5-49

In [5]:
len(answers)

4

In [6]:
for ans in answers:
    print(f"Id: {ans.id}\nTitle: {ans.metadata['title']}\nContent: {ans.page_content}")
    print()

Id: 689584f8-2fd2-483a-92c5-22fb958f0bac
Title: Four AI Agent Strategies That Improve GPT-4 and GPT-3.5 Performance
Content: up tasks and discussing and debating ideas, to come up with better solutions than a single agent would.Next week, I’ll elaborate on these design patterns and offer suggested readings for each.Keep learning!AndrewRead "Agentic Design Patterns Part 2: Reflection"Read "Agentic Design Patterns Part 3, Tool Use"Read "Agentic Design Patterns Part 4: Planning"Read "Agentic Design Patterns Part 5: Multi-Agent Collaboration"ShareSubscribe to The BatchStay updated with weekly AI News and Insights

Id: 5259b3a0-4ff5-491d-b7f4-b18ebfbc199b
Title: Agentic Design Patterns Part 4: Planning
Content: "Agentic Design Patterns Part 1: Four AI agent strategies that improve GPT-4 and GPT-3.5 performance"Read "Agentic Design Patterns Part 2: Reflection" Read "Agentic Design Patterns Part 3: Tool Use"Read "Agentic Design Patterns Part 5: Multi-Agent Collaboration"ShareSubscribe to The 

### Check document relevancy

In [7]:
## Data Model
class GradeDocuments(BaseModel):
    """Binary score for relevance on retrieved documents."""
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )
    answers: List[str]
    
## LLM with structured output
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document:  \n\n {document} \n\n User question: {question}"),
    ]
)
retrieval_grader = grade_prompt | structured_llm_grader

## Filter out non-relevant docs

In [8]:
docs_to_use = []
for ans in answers:
    print(ans.page_content, '\n', '-'*50)
    res = retrieval_grader.invoke({"document":ans.page_content, "question":question})
    print(res,'\n')
    if res.binary_score == 'yes':
        docs_to_use.append(ans)

up tasks and discussing and debating ideas, to come up with better solutions than a single agent would.Next week, I’ll elaborate on these design patterns and offer suggested readings for each.Keep learning!AndrewRead "Agentic Design Patterns Part 2: Reflection"Read "Agentic Design Patterns Part 3, Tool Use"Read "Agentic Design Patterns Part 4: Planning"Read "Agentic Design Patterns Part 5: Multi-Agent Collaboration"ShareSubscribe to The BatchStay updated with weekly AI News and Insights 
 --------------------------------------------------


binary_score='yes' answers=['Agentic Design Patterns Part 2: Reflection', 'Agentic Design Patterns Part 3, Tool Use', 'Agentic Design Patterns Part 4: Planning', 'Agentic Design Patterns Part 5: Multi-Agent Collaboration'] 

"Agentic Design Patterns Part 1: Four AI agent strategies that improve GPT-4 and GPT-3.5 performance"Read "Agentic Design Patterns Part 2: Reflection" Read "Agentic Design Patterns Part 3: Tool Use"Read "Agentic Design Patterns Part 5: Multi-Agent Collaboration"ShareSubscribe to The BatchStay updated with weekly AI News and Insights delivered to your inboxCoursesThe BatchCommunityCareersAbout 
 --------------------------------------------------
binary_score='yes' answers=['Agentic Design Patterns Part 1: Four AI agent strategies that improve GPT-4 and GPT-3.5 performance', 'Agentic Design Patterns Part 2: Reflection', 'Agentic Design Patterns Part 3: Tool Use', 'Agentic Design Patterns Part 5: Multi-Agent Collaboration'] 

"Agentic Design Patterns Part 3: Tool Use"

## Generate Result

In [9]:
## Prompt
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. 
Use three-to-five sentences maximum and keep the answer concise."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>"),
    ]
)

## llm
llm = ChatGroq(model_name='Gemma2-9b-it', temperature=0)

## Post processing
def format_docs(answors):
    return "\n".join(f'<doc{i+1}>: Title:{doc.metadata['title']}\nSource:{doc.metadata['source']}\nContent:{doc.page_content}</doc{i+1}>\n' for i, doc in enumerate(answers))

## Chain
rag_chain = prompt | llm | StrOutputParser() 

## Invoke
generation = rag_chain.invoke({"documents":format_docs(docs_to_use), "question":question})

In [10]:
print(generation)

The article outlines five agentic design patterns:  Reflection, Tool Use, Planning, Multi-Agent Collaboration, and  Four AI agent strategies.  These patterns enhance the capabilities of AI agents, allowing them to perform more complex tasks and interact with their environment in sophisticated ways. 





## Check for Hallucinations

In [12]:
## Data model
class GradeHallucinations (BaseModel):
    """Binary score for hallucination present in generated answer."""
    binary_score: str = Field(..., description="Answer contains hallucination, 'yes' or 'no'")

## Structured llm
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

## Prompt 
system =  """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
    Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""

hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Answer: {answer} \n\n Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: {question}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader

response = hallucination_grader.invoke

## Highlight Used Docs

In [23]:
from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

## Data Model
class HighlightDocuments(BaseModel):
    """Return the specific part of a document used for answering the question."""
    id: List[str] = Field(..., description="The id of the document used for answering the question.")
    title: List[str] = Field(..., description="List of titles of the documents used for answering the question.")
    source: List[str] = Field(..., description="List of sources of the documents used for answering the question.")
    segment: List[str] = Field(..., description="The specific part of the document used for answering the question.")
    
## Custom parser for Pydantic v2
class CustomPydanticOutputParser(PydanticOutputParser):
    def get_format_instructions(self) -> str:
        schema = self.pydantic_object.model_json_schema()
        return f"""The output should be formatted as a JSON instance that conforms to the JSON schema below.

{schema}

As an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}
the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Please ensure the output is a valid JSON object with the correct schema and not a string representation of it."""
    
# Use the custom parser
parser = CustomPydanticOutputParser(pydantic_object=HighlightDocuments)

# Prompt
system = """You are an advanced assistant for document search and retrieval. You are provided with the following:
1. A question.
2. A generated answer based on the question.
3. A set of documents that were referenced in generating the answer.

Your task is to identify and extract the exact inline segments from the provided documents that directly correspond to the content used to 
generate the given answer. The extracted segments must be verbatim snippets from the documents, ensuring a word-for-word match with the text 
in the provided documents.

Ensure that:
- (Important) Each segment is an exact match to a part of the document and is fully contained within the document text.
- The relevance of each segment to the generated answer is clear and directly supports the answer provided.
- (Important) If you didn't used the specific document don't mention it.

Used documents: <docs>{documents}</docs> \n\n User question: <question>{question}</question> \n\n Generated answer: <answer>{generation}</answer>

<format_instruction>
{format_instructions}
</format_instruction>
"""

prompt = PromptTemplate(
    template=system,
    input_variables=["documents", "question", "generation"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Chain
doc_lookup = prompt | llm | parser

# Run
lookup_response = doc_lookup.invoke({"documents": format_docs(docs_to_use), "question": question, "generation": generation})

In [24]:
for id, title, source, segment in zip(lookup_response.id, lookup_response.title, lookup_response.source, lookup_response.segment):
    print(f"ID: {id}\nTitle: {title}\nSource: {source}\nText Segment: {segment}\n")

ID: doc1
Title: Four AI Agent Strategies That Improve GPT-4 and GPT-3.5 Performance
Source: https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io
Text Segment: Next week, I’ll elaborate on these design patterns and offer suggested readings for each.Keep learning!AndrewRead "Agentic Design Patterns Part 2: Reflection"Read "Agentic Design Patterns Part 3, Tool Use"Read "Agentic Design Patterns Part 4: Planning"Read "Agentic Design Patterns Part 5: Multi-Agent Collaboration"ShareSubscribe to The BatchStay updated with weekly AI News and Insights

